# Analysis of Cohort 1 Samples
----------------------------------------------------------------------------------


## Long-Read Sequencing of CHOP Cancer Fusion Panel Short-Read Libraries - GF Filtering Criteria Pipeline: 

### Description:
This interactive Jupyter Notebook provides the comprehensive analysis pipeline for ONT long-read sequencing of short-read libraries previously processed through the CHOP Cancer Fusion Panel, with a focus on identifying gene fusions (GFs). 

This workflow incorporates GF detection and filtering criteria to help prioritize biologically and clinically relevant GFs. After each filtering step, there is a count of the number of GFs in each sample remaining. The notebook also incorporates cross-referencing with external databases, and a summary of key findings.


### *This notebook can be used for the Analysis and Filtering of Cohort 1 Subset Samples - the Input and Output paths need to updated


----------------------------------------------------------------------------------
## Table of Contents

### 1. Set Inputs
- **a.** Import Libraries and Pandas Display Options  
- **b.** GENCODE and Genome Reference Files  
- **c.** Set Sample Names List 


### 2. Offline Scripts
- **a.** Redo Basecalling
- **b.** Alignment to GRCh38 Human Reference Genome
- **c.** GF Detection Programs (LongGF, JAFFAL, FusionSeeker)
- **d.** Generate GF Summary of Detected GFs (01_Combine_Fusion_Program_Outputs_nb.ipynb)


### 3. Gene Fusion Detection Results & Filtering Criteria: Input Summaries of LongGF, JAFFAL, and FusionSeeker Outputs  
- **a.** Keep GFs involving CHOP Cancer Fusion Panel and Cancer Census Genes  
- **b.** Keep Fusions with More than 2 Reads (10 reads for Cohort 1 Subset Samples)
- **c.** Remove Fusions with Unrelated Genes
- **d.** Keep Fusions with Consistent Strandness   

### 4. Fusion Annotation
- **a.** Cross-Referencing Detected Fusions with Online Fusion Databases  
- **b.** Remove Recurrent Fusions in More Than a Certain Percentage of the Samples (Excluding those found in Online Fusion Databases) 


### 5. Results Summary
- **a.** Summary of Results  


# 1. Set Inputs 

In [ ]:
import pandas as pd
import numpy as np
import pyensembl
import pickle
import sys
import os
import re
import matplotlib.pyplot as plt

# Pandas Display Options for better readability
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# File paths and parameters for GENCODE Reference and Annotation
# Downloaded: https://www.gencodegenes.org/human/release_46.html (using release preference)
# basic annotation GTF, transcript fasta, and protein fasta

reference_name = 'GRCh38'
annotation_name = 'gencode'
annotation_version = 'v46' # recommended

# File paths for GTF, transcript, and protein fasta files
gtf_path = '/path/to/gencode.v46.basic.annotation.gtf.gz'
transcript_fasta_path = '/path/to/gencode.v46.transcripts.fa.gz'
protein_fasta_path = '/path/to/gencode.v46.pc_transcripts.fa.gz'

# Cache directory path for downloading and storing files
cache_directory_path = '/path/to/gencode_v46_downloads_cache/'

### 
# GFF3 v46 Annotation File for Reference Lookups
gff3_input_annotation = '/path/to/gencode.v46.basic.annotation.gff3'

### 
# Set names of samples included in analysis
sample_names = ['sample3']
print("Sample Names: \n", sample_names)

###
# Set path to your summary of all fusions file
input_sum_GF_sample_path = '/path/to/folder/combined_fusion_calls/'

###
# Set Path to CHOP Cancer Fusion Panel and Cancer Census Gene Lists
chop_panel_genes_path = '/path/to/CHOP_Fusion_Panel_genes.txt'
cancer_census_genes_path = '/path/to/Cosmic_CancerGeneCensus_v101_GRCh38.tsv'

###
# Set Read Support Threshold 
set_read_support = 2 # change based on read support needs

###
# Set Path to Literature Reference Files 
mitelman_db_gfs = "/path/to/mitelman_db_fusion_names.txt"
chimer_db_gfs = "/path/to/ChimerDB4.0_fusions.txt"
cosmic_db_gfs = "/path/to/cosmic_fusion_genes_cleaned.tsv"

###
# Set Recurrent Percentage to Filter Fusions On
percentage_samples = 0.15

###
# Set Path to Folder to Save Output Summary Files 
filtered_output_save_folder = '/path/to/filtered_fusion_calls/'


In [ ]:
# File paths and parameters for GENCODE Reference and Annotation
# Downloaded: https://www.gencodegenes.org/human/release_46.html (using release preference)
# basic annotation GTF, transcript fasta, and protein fasta

reference_name = 'GRCh38'
annotation_name = 'gencode'
annotation_version = 'v46' # recommended

# File paths for GTF, transcript, and protein fasta files
gtf_path = '/path/to/gencode.v46.basic.annotation.gtf.gz'
transcript_fasta_path = '/path/to/gencode.v46.transcripts.fa.gz'
protein_fasta_path = '/path/to/gencode.v46.pc_transcripts.fa.gz'

# Cache directory path for downloading and storing files
cache_directory_path = '/path/to/gencode_v46_downloads_cache/'


########
reference_name = 'GRCh38'
annotation_name = 'gencode'
annotation_version = 'v46'

# File paths for GTF, transcript, and protein fasta files
gtf_path = '/mnt/isilon/wang_lab/karly/ref_genome_hg38/cleaned_hg38_ref/gencode.v46.basic.annotation.gtf.gz'
transcript_fasta_path = '/mnt/isilon/wang_lab/karly/ref_genome_hg38/cleaned_hg38_ref/gencode.v46.transcripts.fa.gz'
protein_fasta_path = '/mnt/isilon/wang_lab/karly/ref_genome_hg38/cleaned_hg38_ref/gencode.v46.pc_transcripts.fa.gz'

# Cache directory path for downloading and storing files
cache_directory_path = '/mnt/isilon/wang_lab/karly/ref_genome_hg38/cleaned_hg38_ref/gencode_v46_downloads_cache/'
########




### 

# GFF3 v46 Annotation File for Reference Lookups
gff3_input_annotation = '/path/to/gencode.v46.basic.annotation.gff3'
gff3_input_annotation = '/mnt/isilon/wang_lab/karly/ref_genome_hg38/cleaned_hg38_ref/gencode.v46.basic.annotation.gff3'


### 

# Set names of samples included in analysis
sample_names = ['sample3']
print("Sample Names: \n", sample_names)


###

# Define the path to your summary of all fusions file
input_sum_GF_sample_path = '/mnt/isilon/wang_lab/karly/project/chop_panel_ont_aim1/github_upload/example_chop_fusion_analysis/combined_fusion_calls/'

###
chop_panel_genes_path = '/path/to/CHOP_Fusion_Panel_genes.txt'
cancer_census_genes_path = '/path/to/Cosmic_CancerGeneCensus_v101_GRCh38.tsv'


chop_panel_genes_path = '/mnt/isilon/wang_lab/karly/project/chop_panel_ont_aim1/github_upload/references/CHOP_Fusion_Panel_genes.txt'
cancer_census_genes_path = '/mnt/isilon/wang_lab/karly/project/chop_panel_ont_aim1/github_upload/references/Cosmic_CancerGeneCensus_v101_GRCh38.tsv'


###

set_read_support = 2 # change based on read support needs

###

# Literature Cross Reference 
mitelman_db_gfs = "/path/to/mitelman_db_fusion_names.txt"
chimer_db_gfs = "/path/to/ChimerDB4.0_fusions.txt"
cosmic_db_gfs = "/path/to/cosmic_fusion_genes_cleaned.tsv"


mitelman_db_gfs = '/mnt/isilon/wang_lab/karly/project/chop_panel_ont_aim1/github_upload/references/mitelman_fusion_names.txt'
chimer_db_gfs = '/mnt/isilon/wang_lab/karly/project/chop_panel_ont_aim1/github_upload/references/ChimerDB4.0_fusions.txt'
cosmic_db_gfs = '/mnt/isilon/wang_lab/karly/project/chop_panel_ont_aim1/github_upload/references/cosmic_fusion_genes_cleaned.tsv'

###

# set recurrent percentage to filter fusions from 
percentage_samples = 0.15

###

filtered_output_save_folder = '/mnt/isilon/wang_lab/karly/project/chop_panel_ont_aim1/github_upload/example_chop_fusion_analysis/filtered_fusion_calls/'



In [ ]:
# Initialize the pyensembl Genome object with the defined variables
gencode_data = pyensembl.Genome(
    reference_name=reference_name,
    annotation_name=annotation_name,
    annotation_version=annotation_version,
    gtf_path_or_url=gtf_path,
    transcript_fasta_paths_or_urls=transcript_fasta_path,
    protein_fasta_paths_or_urls=protein_fasta_path,
    cache_directory_path=cache_directory_path,
    copy_local_files_to_cache=False)


# Index the genome data to enable quick access to annotations
print('gencode reference stored')
gencode_data.index()

In [ ]:
# Process GFF3 v46 annotation file for reference lookups, extracting key details.
# Load the GFF3 file into a DataFrame, skipping commented lines and using tab-delimited columns.
annotation_ref_df = pd.read_csv(gff3_input_annotation, sep='\t', comment='#', header=None)

# Rename relevant columns for easier access.
annotation_ref_df.rename(columns={0: 'chrom', 2: 'feature', 3: 'start', 4: 'end', 6: 'strand', 8: 'info'}, inplace=True)

# Extract necessary attributes from the 'info' column using regex.
annotation_ref_df['gene_id'] = annotation_ref_df['info'].str.extract(r'gene_id=([^;]+)')
annotation_ref_df['gene_name'] = annotation_ref_df['info'].str.extract(r'gene_name=([^;]+)')
annotation_ref_df['gene_type'] = annotation_ref_df['info'].str.extract(r'gene_type=([^;]+)')
annotation_ref_df['transcript_id'] = annotation_ref_df['info'].str.extract(r'transcript_id=([^;]+)')
annotation_ref_df['transcript_name'] = annotation_ref_df['info'].str.extract(r'transcript_name=([^;]+)')
annotation_ref_df['transcript_type'] = annotation_ref_df['info'].str.extract(r'transcript_type=([^;]+)')

# Create a mapping of (gene_name, chromosome) to all extracted attributes
gene_name_map = {
    (gene, chrom): (gene_id, strand, gene_type, transcript_id, transcript_name, transcript_type)
    for gene, chrom, gene_id, strand, gene_type, transcript_id, transcript_name, transcript_type in zip(
        annotation_ref_df['gene_name'], annotation_ref_df['chrom'], annotation_ref_df['gene_id'], annotation_ref_df['strand'], annotation_ref_df['gene_type'],
        annotation_ref_df['transcript_id'], annotation_ref_df['transcript_name'], annotation_ref_df['transcript_type']
    )
}

# Function to retrieve gene-related attributes
def get_gene_name_map(gene_name, contig):
    return gene_name_map.get((gene_name, contig), (None, None, None, None, None, None))

# Confirm that the GFF3 file has been processed and the DataFrame is ready for use.
print('GFF3 file is stored!')

# Display the DataFrame for verification.
annotation_ref_df.head()
 

# 2. Offline Scripts
- **a.** Redo Basecalling
- **b.** Alignment to GRCh38 Human Reference Genome
- **c.** LongReadSum Alignment Statistics 
- **d.** GF Detection Programs (LongGF, JAFFAL, FusionSeeker)
- **e.** Generate GF Summary of Detected GFs (Combine Fusion Outputs)


# 3. Gene Fusion Detection Results & Filtering Criteria

In [ ]:
# Initialize an empty list to hold the dataframes for each sample
combined_df_list = []

# Loop through each sample name and process the corresponding files
for sample_number in sample_names:
    LJF_filter_input_file = f'{input_sum_GF_sample_path}/summary_all_fusions.tsv'
    LJF_df = pd.read_csv(LJF_filter_input_file, sep='\t')
    LJF_df['Sample Name'] = sample_number
    combined_df_list.append(LJF_df)
all_fusions_input_df = pd.concat(combined_df_list, ignore_index=True)


# Drop duplicates while keeping the row with the highest 'Supporting Reads'
all_fusions_input_df = (
    all_fusions_input_df
    .sort_values('Supporting Reads', ascending=False)  # Sort by 'Supporting Reads' in descending order
    .drop_duplicates(subset=['Fusion Name', 'Gene 1 Symbol', 'Gene 2 Symbol', 
                              'Gene 1 Breakpoint', 'Gene 2 Breakpoint', 
                              'GF Program', 'Sample Name'], 
                     keep='first'))  # Retain the first occurrence (highest 'Supporting Reads')

print("Total Fusions Detected: ", all_fusions_input_df.shape[0])

# number of unique fusions 
unique_fusions_initial_input = all_fusions_input_df[['Fusion Name', 'Sample Name']].drop_duplicates()
num_unique_fusions_initial_input = len(unique_fusions_initial_input)
print('Unique Gene Fusions Initiall Detected (based on gene name): ', num_unique_fusions_initial_input)

all_fusions_input_df.head()

In [ ]:
# generate a df to keep track of fusion counts while filtering
fusion_counts_while_filtering = unique_fusions_initial_input.groupby('Sample Name')['Fusion Name'].nunique().reset_index()
fusion_counts_while_filtering.columns = ['Sample Name', 'Initial Detected Unique GFs']
fusion_counts_while_filtering

## **a.** Keep GFs with CHOP Cancer Panel Genes and Cancer Census Union of Genes  

In [ ]:
chop_genes_df = pd.read_csv(chop_panel_genes_path, header=None)
chop_genes = set(chop_genes_df[0].dropna())  
print(f"Total number of CHOP genes: {len(chop_genes)}")

###

cancer_census_df = pd.read_csv(cancer_census_genes_path, sep='\t', usecols=['GENE_SYMBOL'])
cc_genes = set(cancer_census_df['GENE_SYMBOL'].dropna())
print('Number of Cancer Census Genes: ', len(cc_genes))


###

# Combine Genes
combined_genes_set = chop_genes.union(cc_genes)
combined_genes_list = set(combined_genes_set)

# Look at overlap
overlap_with_chop = chop_genes.intersection(cc_genes)
print(f"Total number of combined genes: {len(combined_genes_list)}")


In [ ]:
# Filter fusions to those that involve at least one gene in the union of CHOP & Cancer Census genes
chop_cancer_census_gfs = all_fusions_input_df[
    all_fusions_input_df['Gene 1 Symbol'].isin(combined_genes_list) | 
    all_fusions_input_df['Gene 2 Symbol'].isin(combined_genes_list)]

# Note which gene is involved
def check_overlap(genes_set, column_name):
    def check(row):
        gene1_in_panel = row['Gene 1 Symbol'] in genes_set
        gene2_in_panel = row['Gene 2 Symbol'] in genes_set
        if gene1_in_panel and gene2_in_panel:
            return 'Both'
        elif gene1_in_panel:
            return 'Gene 1'
        elif gene2_in_panel:
            return 'Gene 2'
        else:
            return 'None'
    chop_cancer_census_gfs_copy = chop_cancer_census_gfs.copy()
    chop_cancer_census_gfs_copy[column_name] = chop_cancer_census_gfs_copy.apply(check, axis=1)

check_overlap(chop_genes, 'CHOP Fusion Panel Overlap')
check_overlap(cc_genes, 'Cancer Census Overlap')

print(f"Number of fusions with at least 1 CHOP or Cancer Census gene: {chop_cancer_census_gfs.shape[0]}")
chop_cancer_census_gfs.head()

In [ ]:
# add counts to keep track of fusion counts while filtering
fusion_counts_chop_filter1 = chop_cancer_census_gfs.groupby('Sample Name')['Fusion Name'].nunique().reset_index()

# merge 
fusion_counts_while_filtering = fusion_counts_while_filtering.merge(fusion_counts_chop_filter1, on='Sample Name', how='left')
fusion_counts_while_filtering.rename(columns={'Fusion Name': 'Filter on CHOP and Cancer Census Genes'}, inplace=True)
fusion_counts_while_filtering['Filter on CHOP and Cancer Census Genes'] = fusion_counts_while_filtering['Filter on CHOP and Cancer Census Genes'].fillna(0).astype(int)

fusion_counts_while_filtering

## **b.** Keep Fusions with More than 2 Reads

In [ ]:
# Remove those with less than 2 supporting reads
read_filter_df = chop_cancer_census_gfs[chop_cancer_census_gfs['Supporting Reads'] >= set_read_support]

# Number of entries removed
num_removed_read_filter = chop_cancer_census_gfs.shape[0] - read_filter_df.shape[0]

print(f'Number of fusions with {set_read_support} or more supporting reads: {read_filter_df.shape[0]}')
print("Number of fusions removed:", num_removed_read_filter)
read_filter_df.head()

In [ ]:
# add counts to keep track of fusion counts while filtering
fusion_counts_read_filter2 = read_filter_df.groupby('Sample Name')['Fusion Name'].nunique().reset_index()

# merge 
fusion_counts_while_filtering = fusion_counts_while_filtering.merge(fusion_counts_read_filter2, on='Sample Name', how='left')
fusion_counts_while_filtering.rename(columns={'Fusion Name': 'Filter on Read Support'}, inplace=True)
fusion_counts_while_filtering['Filter on Read Support'] = fusion_counts_while_filtering['Filter on Read Support'].fillna(0).astype(int)

fusion_counts_while_filtering

## **c.** Remove Fusions with Unrelated Genes

In [ ]:
# Remove Pseudogenes

# Filter rows where 'feature' column contains the word 'gene'
gene_annotation_ref_df = annotation_ref_df[annotation_ref_df['feature'] == 'gene']

# Merge Gene 1 gene type
def merge_gene_type(df, gene_col, gene_annotation_ref_df, suffix):
    merged_df = df.merge(gene_annotation_ref_df[['gene_id', 'gene_type']], 
                         left_on=gene_col, 
                         right_on='gene_id', 
                         how='left')
    merged_df[f'{suffix} Gene Type'] = merged_df['gene_type'].fillna('Unknown')
    merged_df.drop(columns=['gene_type', 'gene_id'], inplace=True)
    return merged_df

# Merge Gene 1 data
gene1_merged = merge_gene_type(read_filter_df, 'Gene 1 ID', gene_annotation_ref_df, 'Gene 1')

# Merge Gene 2 data
filtered_gene_types_added = merge_gene_type(gene1_merged, 'Gene 2 ID', gene_annotation_ref_df, 'Gene 2')

# Filter for protein-coding genes and exclude pseudogenes
filtered_gene_types_protein_coding = filtered_gene_types_added[
    (filtered_gene_types_added['Gene 1 Gene Type'] == 'protein_coding') | 
    (filtered_gene_types_added['Gene 2 Gene Type'] == 'protein_coding')]

filtered_gene_types_protein_coding = filtered_gene_types_protein_coding[
    ~filtered_gene_types_protein_coding['Gene 1 Gene Type'].str.contains('pseudogene', na=False) & 
    ~filtered_gene_types_protein_coding['Gene 2 Gene Type'].str.contains('pseudogene', na=False)]


# Additionally, remove GFs: # where either 'Gene 1 Breakpoint' or 'Gene 2 Breakpoint' contains 'chrM'
# or 'Gene 1 Symbol' or 'Gene 2 Symbol' starts with 'RPL', 'MRP', 'RPS', or 'HLA'

# Patterns to filter out
chromosome_pattern = ['chrM']
ribo_pattern = r'^(RP|RPL|MRP|RPS)'  # Pattern for ribosomal proteins
hla_pattern = r'^(HLA)'  # Pattern for HLA genes

# Perform the combined filtering to remove unwanted GFs
filtered_no_unrelated = filtered_gene_types_protein_coding[
    ~filtered_gene_types_protein_coding['Gene 1 Breakpoint'].str.contains('|'.join(chromosome_pattern), na=False) & 
    ~filtered_gene_types_protein_coding['Gene 2 Breakpoint'].str.contains('|'.join(chromosome_pattern), na=False) & 
    ~filtered_gene_types_protein_coding['Gene 1 Symbol'].str.match(ribo_pattern) & 
    ~filtered_gene_types_protein_coding['Gene 2 Symbol'].str.match(ribo_pattern) & 
    ~filtered_gene_types_protein_coding['Gene 1 Symbol'].str.match(hla_pattern) & 
    ~filtered_gene_types_protein_coding['Gene 2 Symbol'].str.match(hla_pattern)
]

# Display the filtered results
print("Number of fusions remaining:", filtered_no_unrelated.shape[0])
print("Number of fusions removed:", read_filter_df.shape[0] - filtered_no_unrelated.shape[0])
filtered_no_unrelated

In [ ]:
# add counts to keep track of fusion counts while filtering
fusion_counts_gene_type_filter3 = filtered_no_unrelated.groupby('Sample Name')['Fusion Name'].nunique().reset_index()

# merge 
fusion_counts_while_filtering = fusion_counts_while_filtering.merge(fusion_counts_gene_type_filter3, on='Sample Name', how='left')
fusion_counts_while_filtering.rename(columns={'Fusion Name': 'Filter on Unrelated Genes'}, inplace=True)
fusion_counts_while_filtering['Filter on Unrelated Genes'] = fusion_counts_while_filtering['Filter on Unrelated Genes'].fillna(0).astype(int)

fusion_counts_while_filtering

##  **d.** Keep Fusions with Consistent Strandness   

In [ ]:
# Remove those with inconsistent strandness 
consistent_filter_df = filtered_no_unrelated[filtered_no_unrelated['Strand Comparison'] == 'Consistent']

# Calculate how many entries were removed
num_removed_consistent_filter = filtered_no_unrelated.shape[0] - consistent_filter_df.shape[0]

# Display the results
print('Number of fusions with Consistent Strandness:', consistent_filter_df.shape[0])
print("Number of fusions removed:", num_removed_consistent_filter)
consistent_filter_df.head()


In [ ]:
# add counts to keep track of fusion counts while filtering
fusion_counts_consistent_filter4 = consistent_filter_df.groupby('Sample Name')['Fusion Name'].nunique().reset_index()

# merge 
fusion_counts_while_filtering = fusion_counts_while_filtering.merge(fusion_counts_consistent_filter4, on='Sample Name', how='left')
fusion_counts_while_filtering.rename(columns={'Fusion Name': 'Filter on Consistent Strandness'}, inplace=True)
fusion_counts_while_filtering['Filter on Consistent Strandness'] = fusion_counts_while_filtering['Filter on Consistent Strandness'].fillna(0).astype(int)

fusion_counts_while_filtering

# 4. Fusion Annotation
## **a.** Cross-Referencing Detected Fusions with External Databases 
- Mitelman, COSMIC, and ChimerDB

In [ ]:
# Mitelman DB
mitelman_ref = pd.read_csv(mitelman_db_gfs, delimiter='\t', header=None)
mitelman_ref.columns = ['Fusion Name']
mitelman_ref['Fusion Name'] = mitelman_ref['Fusion Name'].str.strip()

split_fusions = mitelman_ref['Fusion Name'].str.split(':', n=1)  
mitelman_ref[['Gene 1', 'Gene 2']] = pd.DataFrame(split_fusions.tolist(), index=mitelman_ref.index)
mitelman_ref = mitelman_ref.drop(columns=['Fusion Name'])
mitelman_ref.drop_duplicates()


# Chimer DB
chimer_ref = pd.read_csv(chimer_db_gfs, delimiter='\t', header=None)
chimer_ref.columns = ['Fusion Name']
chimer_ref['Fusion Name'] = chimer_ref['Fusion Name'].str.strip()

split_fusions = chimer_ref['Fusion Name'].str.split(':', n=1)  
chimer_ref[['Gene 1', 'Gene 2']] = pd.DataFrame(split_fusions.tolist(), index=chimer_ref.index)
chimer_ref = chimer_ref.drop(columns=['Fusion Name'])
chimer_ref.drop_duplicates()


# Cosmic DB
cosmic_ref = pd.read_csv(cosmic_db_gfs, delimiter='\t', header=None)
cosmic_ref.columns = ['Fusion Name']
cosmic_ref['Fusion Name'] = cosmic_ref['Fusion Name'].str.strip()

split_fusions = cosmic_ref['Fusion Name'].str.split(':', n=1)  
cosmic_ref[['Gene 1', 'Gene 2']] = pd.DataFrame(split_fusions.tolist(), index=cosmic_ref.index)
cosmic_ref = cosmic_ref.drop(columns=['Fusion Name'])
cosmic_ref.drop_duplicates()


# Combine All Fusion Online Database References
all_fusions = pd.concat([mitelman_ref, chimer_ref, cosmic_ref])
unique_fusions = all_fusions.drop_duplicates(subset=['Gene 1', 'Gene 2'])


# Convert the reference files into sets for faster lookup
mitelman_set = set(mitelman_ref.apply(lambda row: f"{row['Gene 1']}:{row['Gene 2']}", axis=1).dropna())
chimer_set = set(chimer_ref.apply(lambda row: f"{row['Gene 1']}:{row['Gene 2']}", axis=1).dropna())
cosmic_set = set(cosmic_ref.apply(lambda row: f"{row['Gene 1']}:{row['Gene 2']}", axis=1).dropna())

In [ ]:
# Create a function to check if fusion is present in any reference file
def check_fusion_in_references(row):
    fusion1 = f"{row['Gene 1 Symbol']}:{row['Gene 2 Symbol']}"
    fusion2 = f"{row['Gene 2 Symbol']}:{row['Gene 1 Symbol']}"  # Reverse orientation
    result = []

    # Check against each reference set
    if fusion1 in mitelman_set or fusion2 in mitelman_set:
        result.append('Mitelman Reference')
    if fusion1 in chimer_set or fusion2 in chimer_set:
        result.append('Chimer Reference')
    if fusion1 in cosmic_set or fusion2 in cosmic_set:
        result.append('COSMIC Reference')

    # Return the result as a comma-separated string
    return ', '.join(result) if result else 'No match'

# Apply the function to each row of the dataframe
filter_df_literature = consistent_filter_df.copy()
filter_df_literature['Literature Cross Reference'] = filter_df_literature.apply(check_fusion_in_references, axis=1)
known_gfs_detected_df = filter_df_literature[filter_df_literature['Literature Cross Reference'] != 'No match']


# Show overview of known fusions by sample 
unique_gfs_per_sample = (
    known_gfs_detected_df.groupby('Sample Name')['Fusion Name']
    .apply(lambda x: ", ".join(sorted(set(x))))  
    .reset_index(name='Unique Gene Fusions'))

# Add count of unique fusions per sample
unique_gfs_per_sample.insert(1, 'Number of Unique Known GFs', unique_gfs_per_sample['Unique Gene Fusions'].apply(lambda x: x.count(",") + 1 if x else 0))
unique_gfs_per_sample.sort_values(by='Sample Name', inplace=True)


# Get the total number of unique fusions
num_unique_fusions = len(known_gfs_detected_df['Fusion Name'].value_counts())
num_unique_samples = len(known_gfs_detected_df['Sample Name'].value_counts())
print(f"Total unique fusions: {num_unique_fusions} in {num_unique_samples} samples.")
display(unique_gfs_per_sample)


# look at those reported in literature only .. just annotation no filtering!
display(filter_df_literature)

In [ ]:
# add counts to keep track of fusion counts while filtering
fusion_counts_known_ref_filter5 = known_gfs_detected_df.groupby('Sample Name')['Fusion Name'].nunique().reset_index()

# merge 
fusion_counts_while_filtering = fusion_counts_while_filtering.merge(fusion_counts_known_ref_filter5, on='Sample Name', how='left')
fusion_counts_while_filtering.rename(columns={'Fusion Name': 'GFs in Fusion Reference Databases'}, inplace=True)
fusion_counts_while_filtering['GFs in Fusion Reference Databases'] = fusion_counts_while_filtering['GFs in Fusion Reference Databases'].fillna(0).astype(int)

fusion_counts_while_filtering

##  **b.** Remove Recurrent Fusions in More Than a Certain Percentage of the Samples  

In [ ]:
print("Recurrent Fusions Removed Filter: ")

# Count unique samples each fusion appears in
fusion_sample_counts = filter_df_literature.groupby(['Fusion Name', 'Gene 1 Symbol', 'Gene 2 Symbol'])['Sample Name'].nunique()

# Calculate threshold and filter based on occurrence
total_samples = filter_df_literature['Sample Name'].nunique()

threshold = total_samples * percentage_samples

recurrent_fusions = fusion_sample_counts[fusion_sample_counts > threshold]
recurrent_fusions_df = recurrent_fusions.reset_index()
recurrent_fusions_df.rename(columns={'Sample Name': 'Sample Count'}, inplace=True)

# Output
print(f"Total samples: {total_samples}, using a {percentage_samples*100}% threshold: {threshold} occurrences within a sample")
print("Recurrent fusions")
display(recurrent_fusions_df)

# Show original entries 
original_recurrent_entries_df = filter_df_literature[
    filter_df_literature[['Fusion Name', 'Gene 1 Symbol', 'Gene 2 Symbol']].apply(
        tuple, axis=1
    ).isin(recurrent_fusions_df[['Fusion Name', 'Gene 1 Symbol', 'Gene 2 Symbol']].apply(tuple, axis=1))
]
print(f"Original entries matching the recurrent fusions: {original_recurrent_entries_df.shape[0]}")




### EXCLUDE: Fusions Identified in Online Databases from Recurrent Removal ###

# Identify fusions in literature (direct match or reciprocal)
if 'Literature Cross Reference' in filter_df_literature.columns:
    literature_fusions = set(
        filter_df_literature[
            filter_df_literature['Literature Cross Reference'] != 'No match'
        ][['Gene 1 Symbol', 'Gene 2 Symbol']].apply(tuple, axis=1)
    )
else:
    literature_fusions = set()

if 'Literature Reciprocal Fusion' in filter_df_literature.columns:
    reciprocal_literature_fusions = set(
        filter_df_literature[
            filter_df_literature['Literature Reciprocal Fusion'] == 'Yes'
        ][['Gene 1 Symbol', 'Gene 2 Symbol']].apply(lambda x: (x[1], x[0]), axis=1)
    )
else:
    reciprocal_literature_fusions = set()

# Combine both sets
literature_fusions |= reciprocal_literature_fusions

# Create a set of recurrent fusion pairs (both orientations)
recurrent_fusion_pairs = set(
    list(zip(recurrent_fusions.index.get_level_values('Gene 1 Symbol'), recurrent_fusions.index.get_level_values('Gene 2 Symbol'))) +
    list(zip(recurrent_fusions.index.get_level_values('Gene 2 Symbol'), recurrent_fusions.index.get_level_values('Gene 1 Symbol')))
)

# Exclude fusions in literature from removal
filtered_fusion_pairs = recurrent_fusion_pairs - literature_fusions

# Filter out only the **non-literature** recurrent fusions
recurrent_filter_df = filter_df_literature[
    ~filter_df_literature[['Gene 1 Symbol', 'Gene 2 Symbol']].apply(tuple, axis=1).isin(filtered_fusion_pairs)
]

# Results
rows_removed = filter_df_literature.shape[0] - recurrent_filter_df.shape[0]
print(f"\nNumber of rows removed: {rows_removed}")
print(f"Recurrent GFs Removed, Remaining GFs: {recurrent_filter_df.shape}")

# Show which recurrent fusions were found in literature and not removed
literature_recurrent_fusions = recurrent_fusion_pairs & literature_fusions
literature_recurrent_fusions_df = filter_df_literature[
    filter_df_literature[['Gene 1 Symbol', 'Gene 2 Symbol']].apply(tuple, axis=1).isin(literature_recurrent_fusions)
]

print("\nRecurrent fusions found in literature and not removed:")
print("\n".join(literature_recurrent_fusions_df['Fusion Name'].unique()))

recurrent_filter_df.head()


In [ ]:
# Add counts to keep track of fusion counts while filtering
fusion_counts_recurrent_GF_filter8 = recurrent_filter_df.groupby('Sample Name')['Fusion Name'].nunique().reset_index()
fusion_counts_while_filtering = fusion_counts_while_filtering.merge(fusion_counts_recurrent_GF_filter8, on='Sample Name', how='left')

recurrent_col_name = f'Filter on Recurrent in {percentage_samples*100}% of Samples'

fusion_counts_while_filtering.rename(
    columns={'Fusion Name': recurrent_col_name}, 
    inplace=True)

# Fill NaN values and convert to int
fusion_counts_while_filtering[recurrent_col_name] = fusion_counts_while_filtering[recurrent_col_name].fillna(0).astype(int)

fusion_counts_while_filtering

# 5. Results Summary

In [ ]:
# Save the Filtered Fusion Entries
output_file_path = f'{filtered_output_save_folder}summary_filtered_output.tsv'
recurrent_filter_df.to_csv(output_file_path, sep='\t', index=False)
print(f"File saved to: {output_file_path}")


# Save the Counts of Fusions after each Filtering Step 
# Specify the output file path
output_file_path_counts = f'{filtered_output_save_folder}summary_filtered_output_counts.tsv'
fusion_counts_while_filtering.to_csv(output_file_path_counts, sep='\t', index=False)
print(f"File saved to: {output_file_path_counts}")

In [ ]:
# look over counts in df: fusion_counts_while_filtering

# looking at 'Unique GFs' -- based on gene names - not variable by breakpoint
# Drop duplicates based on 'Fusion Name' and 'Sample Name' while keeping the row with the highest 'Supporting Reads'
recurrent_filter_df_UNIQUE = recurrent_filter_df.loc[
    recurrent_filter_df.groupby(['Fusion Name', 'Sample Name'])['Supporting Reads'].idxmax()]


# Total number of GF entries remaining
print("Results Summary")
print("-" * 50)
print(f"Total number of GF entries remaining: {recurrent_filter_df_UNIQUE.shape[0]}")
print("-" * 50)


final_summary_df = fusion_counts_while_filtering

# Number of Entries without any GFs remaining:
# Define the dynamic column name based on percentage_samples
recurrent_col_name = f'Filter on Recurrent in {percentage_samples*100}% of Samples'

# Number of Entries without any GFs remaining:
no_entries_remaining = final_summary_df[final_summary_df[recurrent_col_name] == 0]
print(f"Number of samples with no fusions: {no_entries_remaining.shape[0]}", no_entries_remaining['Sample Name'].unique())

# Number of Entries with exactly ONE GF remaining:
one_entry_remaining = final_summary_df[final_summary_df[recurrent_col_name] == 1]
print(f"Number of samples with only one fusion: {one_entry_remaining.shape[0]}", one_entry_remaining['Sample Name'].unique())

# Number of Entries with MORE THAN ONE GF remaining:
more_than_one_entry_remaining = final_summary_df[final_summary_df[recurrent_col_name] > 1]
print(f"Number of samples with more than one fusion: {more_than_one_entry_remaining.shape[0]}", more_than_one_entry_remaining['Sample Name'].unique())

print("-" * 50)



# 
print('Looking over fusion positive samples, below: ')
final_summary_df = fusion_counts_while_filtering[~fusion_counts_while_filtering['Sample Name'].isin(['sample21', 'sample25'])]


# get min, max, and mean number of fusions across samples
min_fusions = int(final_summary_df[recurrent_col_name].min())
print(f"Minimum number of fusions: {min_fusions}")

max_fusions = int(final_summary_df[recurrent_col_name].max())
print(f"Maximum number of fusions: {max_fusions}")

mean_fusions = int(final_summary_df[recurrent_col_name].mean())
print(f"Mean number of fusions: {mean_fusions}")
print("-" * 50)

# Count of GF Program entries
gf_program_counts = recurrent_filter_df_UNIQUE['GF Program'].value_counts()
print("GF Program Counts:")
print(gf_program_counts)
print("-" * 50)

# Average number of supporting reads over remaining fusions:
mean_fusions_remaining = int(recurrent_filter_df_UNIQUE['Supporting Reads'].mean())
print(f"Mean number of Reads over Remaining Fusions: {mean_fusions_remaining}")
print("-" * 100)


#######################################################################################
# Looking over the NON-unique entries -- to see if reported in multiple GF Programs 
# may have varying breakpoint locations 
#######################################################################################

# Overlap in 3 programs (LongGF, JAFFAL, FusionSeeker)
gf_programs = ['LongGF', 'JAFFAL', 'FusionSeeker']
filtered_df = recurrent_filter_df[recurrent_filter_df['GF Program'].isin(gf_programs)]

# Group by fusion, gene symbols, and sample, and count unique GF Programs
program_count = filtered_df.groupby(['Fusion Name', 'Gene 1 Symbol', 'Gene 2 Symbol', 'Sample Name'])['GF Program'].nunique()

# Filter only those fusions detected in all three programs
three_program_count = program_count[program_count == 3].reset_index()

# Merge with original DataFrame to get supporting read count
three_program_count = three_program_count.merge(
    recurrent_filter_df[['Fusion Name', 'Sample Name', 'Supporting Reads']], 
    on=['Fusion Name', 'Sample Name'], 
    how='left')

# Remove duplicates caused by merging and get max supporting reads per fusion
three_program_count = (
    three_program_count.groupby(['Fusion Name', 'Gene 1 Symbol', 'Gene 2 Symbol', 'Sample Name'])
    .agg({'GF Program': 'first', 'Supporting Reads': 'max'})  # Take max supporting reads
    .reset_index())

# Sort by Supporting Reads in descending order
three_program_count.sort_values(by='Supporting Reads', ascending=False, inplace=True)

# Display results
print(f"Number of fusions reported in all 3 programs (LongGF, JAFFAL, FusionSeeker): {three_program_count.shape[0]}")
print("-" * 100)
# display(three_program_count)


#######################################################################################

# Overlap in only 2 programs (LongGF, JAFFAL, FusionSeeker)
print("Groups reported in only 2 programs:")

# Filter fusions detected in exactly 2 programs
two_program_count = program_count[program_count == 2].reset_index()

# Merge to get Supporting Reads from original DataFrame
two_program_count = two_program_count.merge(
    recurrent_filter_df[['Fusion Name', 'Sample Name', 'Supporting Reads']], 
    on=['Fusion Name', 'Sample Name'], 
    how='left')

# Remove duplicates and get max Supporting Reads per fusion
two_program_count = (
    two_program_count.groupby(['Fusion Name', 'Gene 1 Symbol', 'Gene 2 Symbol', 'Sample Name'])
    .agg({'GF Program': 'first', 'Supporting Reads': 'max'})  # Keep max supporting reads
    .reset_index())

# Sort by Supporting Reads in descending order
two_program_count.sort_values(by='Supporting Reads', ascending=False, inplace=True)

# Print summary and display table
print(f"Number of fusions reported in exactly 2 programs (LongGF, JAFFAL, FusionSeeker): {two_program_count.shape[0]}")
print("-" * 100)
# display(two_program_count)

#######################################################################################
